## Setup

In [ ]:
# Load packages
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix, auc, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Get data
vdem_2022_repl = pd.read_csv("https://raw.githubusercontent.com/vdeminstitute/part/main/create-data/output/part-v12.csv")

In [ ]:
# Create training data set (1970 to 2020)
train_data = vdem_2022_repl[vdem_2022_repl['year']<2021]

# Create test data set (2014 to 2020) <-- this follows the baseline paper approach (not good practice!)
test_data_pre = vdem_2022_repl[vdem_2022_repl['year']>2013]
test_data = test_data_pre[test_data_pre['year']<2021]

In [ ]:
# Select features and target variables for train and data sets

X_train = train_data.drop(columns = ["gwcode", "year", "country_name", "country_text_id", "country_id",
             "v2x_regime", "v2x_regime_amb", "any_neg_change", "lagged_v2x_regime_asCharacter", "lagged_v2x_regime_asFactor", "any_neg_change_2yr"], axis=1)
             
# Step above drops several identifier variables not needed for modeling (same approach as baseline)
y_train = train_data.any_neg_change_2yr.values # Target variable

# For test_data
X_test = test_data.drop(columns = ["gwcode", "year", "country_name", "country_text_id", "country_id",
             "v2x_regime", "v2x_regime_amb", "any_neg_change","lagged_v2x_regime_asCharacter", "lagged_v2x_regime_asFactor", "any_neg_change_2yr"], axis=1)
y_test = test_data.any_neg_change_2yr.values # Target variable


## Model 0: Plain Logistic Regression

In [ ]:
# Build model 0
scaler = StandardScaler()
lr0 = LogisticRegression(solver='lbfgs', max_iter=400)
model0 = Pipeline([('standardize', scaler),
                   ('log_reg',lr0)])

In [ ]:
# Fit model 0
model0.fit(X_train, y_train)

Pipeline(steps=[('standardize', StandardScaler()),
                ('log_reg', LogisticRegression(max_iter=400))])

Evaluating training score

In [ ]:
y_train_hat0 = model0.predict(X_train)
y_train_hat_probs0 = model0.predict_proba(X_train)[:,1]

train_accuracy0 = accuracy_score(y_train, y_train_hat0)*100
train_auc_roc0 = roc_auc_score(y_train, y_train_hat_probs0)*100

train_precision0 = precision_score(y_train, y_train_hat0)*100
train_recall0 = recall_score(y_train, y_train_hat0)*100

print('Confusion matrix:\n', confusion_matrix(y_train, y_train_hat0))

print('Training Accuracy: %.4f %%' % train_accuracy0)
print('Training AUC: %.4f %%' % train_auc_roc0)
print('Training Precision: %.4f %%' % train_precision0)
print('Training Recall: %.4f %%' % train_recall0)



Confusion matrix:
 [[7669   29]
 [ 190  133]]
Training Accuracy: 97.2697 %
Training AUC: 96.3032 %
Training Precision: 82.0988 %
Training Recall: 41.1765 %


Evaluating testing score

In [ ]:
y_test_hat0 = model0.predict(X_test)
y_test_hat_probs0 = model0.predict_proba(X_test)[:,1]

test_accuracy0 = accuracy_score(y_test, y_test_hat0)*100
test_auc_roc0 = roc_auc_score(y_test, y_test_hat_probs0)*100

test_precision0 = precision_score(y_test, y_test_hat0)*100
test_recall0 = recall_score(y_test, y_test_hat0)*100

print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat0))

print('Test Accuracy: %.4f %%' % test_accuracy0)
print('Test AUC: %.4f %%' % test_auc_roc0)
print('Test Precision: %.4f %%' % test_precision0)
print('Test Recall: %.4f %%' % test_recall0)

Confusion matrix:
 [[1108    7]
 [  42   26]]
Test Accuracy: 95.8580 %
Test AUC: 94.8444 %
Test Precision: 78.7879 %
Test Recall: 38.2353 %


In [ ]:
# Print classification report
print(classification_report(y_test, y_test_hat0, digits=6))

              precision    recall  f1-score   support

         0.0   0.963478  0.993722  0.978366      1115
         1.0   0.787879  0.382353  0.514851        68

    accuracy                       0.958580      1183
   macro avg   0.875679  0.688037  0.746609      1183
weighted avg   0.953385  0.958580  0.951723      1183



## Model 1: Logistic Regression with Elastic Net Regularization


In [ ]:
# Build model
scaler = StandardScaler()
lr1 = LogisticRegression(solver='saga',penalty='elasticnet', l1_ratio=0.5, max_iter=6000)
model1 = Pipeline([('standardize', scaler),
                   ('log_reg',lr1)])

In [ ]:
# Fit model
model1.fit(X_train, y_train)

Pipeline(steps=[('standardize', StandardScaler()),
                ('log_reg',
                 LogisticRegression(l1_ratio=0.5, max_iter=6000,
                                    penalty='elasticnet', solver='saga'))])

In [ ]:
# Predict y values for test data
logit_pred=model1.predict(X_test)

Evaluate training score

In [41]:
y_train_hat1 = model1.predict(X_train)
y_train_hat_probs1 = model1.predict_proba(X_train)[:,1]

train_accuracy1 = accuracy_score(y_train, y_train_hat1)*100
train_auc_roc1 = roc_auc_score(y_train, y_train_hat_probs1)*100
train_precision1 = precision_score(y_train, y_train_hat1)*100
train_recall1 = recall_score(y_train, y_train_hat1)*100

print('Confusion matrix:\n', confusion_matrix(y_train, y_train_hat1))

print('Training Accuracy: %.4f %%' % train_accuracy1)
print('Training AUC: %.4f %%' % train_auc_roc1)
print('Training Precision: %.4f %%' % train_precision1)
print('Training Recall: %.4f %%' % train_recall1)


Confusion matrix:
 [[7669   29]
 [ 203  120]]
Training Accuracy: 97.1076 %
Training AUC: 95.9706 %
Training Precision: 80.5369 %
Training Recall: 37.1517 %


Evaluating testing score

In [42]:
y_test_hat1 = model1.predict(X_test)
y_test_hat_probs1 = model1.predict_proba(X_test)[:,1]

test_accuracy1 = accuracy_score(y_test, y_test_hat1)*100
test_auc_roc1 = roc_auc_score(y_test, y_test_hat_probs1)*100
test_precision1 = precision_score(y_test, y_test_hat1)*100
test_recall1 = recall_score(y_test, y_test_hat1)*100

print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat1))

print('Testing Accuracy: %.4f %%' % test_accuracy1)
print('Testing AUC: %.4f %%' % test_auc_roc1)
print('Testing Precision: %.4f %%' % test_precision1)
print('Testing Recall: %.4f %%' % test_recall1)

Confusion matrix:
 [[1108    7]
 [  44   24]]
Testing Accuracy: 95.6889 %
Testing AUC: 94.3656 %
Testing Precision: 77.4194 %
Testing Recall: 35.2941 %


In [ ]:
# Print classification report
print(classification_report(y_test, y_test_hat1, digits=6))

              precision    recall  f1-score   support

         0.0   0.961806  0.993722  0.977503      1115
         1.0   0.774194  0.352941  0.484848        68

    accuracy                       0.956889      1183
   macro avg   0.868000  0.673332  0.731176      1183
weighted avg   0.951021  0.956889  0.949185      1183



In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#tuning, one_tune_iteration, tune_grid
#14-fold cv
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

# Use grid search to tune the parameters:

    parametersGrid = {"max_iter": [2:151], 
                      "alpha": (1)), 
                      "l1_ratio": np.arange(0.0)} 


    eNet = ElasticNet()
    grid = GridSearchCV(eNet, parametersGrid, scoring='accuracy', cv=14)
    grid.fit(X_train, Y_train)
  

## Model 2: Random Forest

In [ ]:
#Create a Gaussian Classifier
model_rf = RandomForestClassifier(n_estimators=2000)

#Train the model using the training sets
model_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000)

Evaluating training score

In [45]:
y_train_hat_rf = model_rf.predict(X_train)
y_train_hat_probs_rf = model_rf.predict_proba(X_train)[:,1]

train_accuracy_rf = accuracy_score(y_train, y_train_hat_rf)*100
train_auc_roc_rf = roc_auc_score(y_train, y_train_hat_probs_rf)*100
train_precision_rf = precision_score(y_train, y_train_hat_rf)*100
train_recall_rf = recall_score(y_train, y_train_hat_rf)*100

print('Confusion matrix:\n', confusion_matrix(y_train, y_train_hat_rf))

print('Training Accuracy: %.4f %%' % train_accuracy_rf)
print('Training AUC: %.4f %%' % train_auc_roc_rf)
print('Training Precision: %.4f %%' % train_precision_rf)
print('Training Recall: %.4f %%' % train_recall_rf)

Confusion matrix:
 [[7698    0]
 [   0  323]]
Training Accuracy: 100.0000 %
Training AUC: 100.0000 %
Training Precision: 100.0000 %
Training Recall: 100.0000 %


Evaluating testing score

In [51]:
y_test_hat_rf = model_rf.predict(X_test)
y_test_hat_probs_rf = model_rf.predict_proba(X_test)[:,1]

test_accuracy_rf = accuracy_score(y_test, y_test_hat_rf)*100
test_auc_roc_rf = roc_auc_score(y_test, y_test_hat_probs_rf)*100
test_precision_rf = precision_score(y_test, y_test_hat_rf)*100
test_recall_rf = recall_score(y_test, y_test_hat_rf)*100

print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat_rf))

print('testing Accuracy: %.4f %%' % test_accuracy_rf)
print('testing AUC: %.4f %%' % test_auc_roc_rf)
print('testing Precision: %.4f %%' % test_precision_rf)
print('testing Recall: %.4f %%' % train_recall_rf)

Confusion matrix:
 [[1115    0]
 [   0   68]]
testing Accuracy: 100.0000 %
testing AUC: 100.0000 %
testing Precision: 100.0000 %
testing Recall: 100.0000 %


Hyperparameter Tuning Grid Search Test

In [ ]:
param_grid = {'bootstrap': [True],
     'max_depth': [1, 20],
     'max_features': [5, 140],
     #'min_samples_leaf': [3, 5],
     'min_samples_leaf': [0.5, 1],
     'n_estimators': [500, 3099] #Is it looking only at 500 and 3099 or everything inbetween???
    }
     
forest_clf = RandomForestClassifier()

forest_grid_search = GridSearchCV(forest_clf, param_grid, cv=14,
                                  scoring="accuracy",
                                  return_train_score=True,
                                  verbose=True,
                                  n_jobs=-1)

forest_grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


GridSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [1, 8],
                         'max_features': [5, 8], 'min_samples_leaf': [0.5, 1],
                         'n_estimators': [20, 30]},
             return_train_score=True, scoring='accuracy', verbose=True)

In [ ]:
forest_grid_search.best_params_

{'bootstrap': True,
 'max_depth': 8,
 'max_features': 8,
 'min_samples_leaf': 1,
 'n_estimators': 20}

In [ ]:
forest_grid_search.best_estimator_

In [ ]:
forest_grid_search.best_score_

Predict probabilities

In [ ]:
# Predict y values for test data
rf_pred=model_rf.predict(X_test)

After training, check the accuracy using actual and predicted values.

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

## Model 3: Gradient Boosted Forest

In [ ]:
# Build the model
model_xgb = xgb.XGBClassifier(objective ='binary:logistic', n_estimators = 50, learning_rate = 0.25,
                              gamma = 0, max_depth = 8, min_child_weight = 2.5, max_delta_step = 5,
                              subsample = 0.7, colsample_bytree = 0.65, alpha = 0)

In [ ]:
# Fit the model
model_xgb.fit(X_train, y_train)

XGBClassifier(alpha=0, colsample_bytree=0.65, learning_rate=0.25,
              max_delta_step=5, max_depth=8, min_child_weight=2.5,
              n_estimators=50, subsample=0.7)

In [46]:
# Predict on the test set
xgb_pred = model_xgb.predict(X_test)

Evaluating training score

In [47]:
y_train_hat_xgb = model_xgb.predict(X_train)
y_train_hat_probs_xgb = model_xgb.predict_proba(X_train)[:,1]

train_accuracy_xgb = accuracy_score(y_train, y_train_hat_xgb)*100
train_auc_roc_xgb = roc_auc_score(y_train, y_train_hat_probs_xgb)*100
train_precision_xgb = precision_score(y_train, y_train_hat_xgb)*100
train_recall_xgb = recall_score(y_train, y_train_hat_xgb)*100

print('Confusion matrix:\n', confusion_matrix(y_train, y_train_hat_xgb))

print('Training Accuracy: %.4f %%' % train_accuracy_xgb)
print('Training AUC: %.4f %%' % train_auc_roc_xgb)
print('Training Precision: %.4f %%' % train_precision_xgb)
print('Training Recall: %.4f %%' % train_recall_xgb)

Confusion matrix:
 [[7698    0]
 [   9  314]]
Training Accuracy: 99.8878 %
Training AUC: 99.9990 %
Training Precision: 100.0000 %
Training Recall: 97.2136 %


Evaluating testing score

In [49]:
y_test_hat_xgb = model_xgb.predict(X_test)
y_test_hat_probs_xgb = model_xgb.predict_proba(X_test)[:,1]

test_accuracy_xgb = accuracy_score(y_test, y_test_hat_xgb)*100
test_auc_roc_xgb = roc_auc_score(y_test, y_test_hat_probs_xgb)*100
test_precision_xgb = precision_score(y_test, y_test_hat_xgb)*100
test_recall_xgb = recall_score(y_test, y_test_hat_xgb)*100

print('Confusion matrix:\n', confusion_matrix(y_test, y_test_hat_xgb))

print('testing Accuracy: %.4f %%' % test_accuracy_xgb)
print('testing AUC: %.4f %%' % test_auc_roc_xgb)
print('testing Precision: %.4f %%' % test_precision_xgb)
print('testing Recall: %.4f %%' % test_recall_xgb)

Confusion matrix:
 [[1115    0]
 [   2   66]]
testing Accuracy: 99.8309 %
testing AUC: 100.0000 %
testing Precision: 100.0000 %
testing Recall: 97.0588 %


In [ ]:
# Compute the rmse by invoking the mean_sqaured_error function from sklearn's metrics module
rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
print("RMSE: %f" % (rmse))

RMSE: 0.041117


In [ ]:
param_grid = {
    'n_estimators': [2, 150],
    'learning_rate': [0, 1],
    'max_depth': [2, 20],
    }
     
xgb_model = xgb.XGBClassifier()

xgb_grid_search = GridSearchCV(xgb_model, param_grid, cv=14,
                                  scoring="accuracy",
                                  return_train_score=True,
                                  verbose=True,
                                  n_jobs=-1)

xgb_grid_search.fit(X_train, y_train)

## Ensemble Model

In [50]:
# Estimate predicted probabilities for each model
logit_pred_probabilties = model1.predict_proba(X_train)[:,1]
rf_pred_probabilties = model_rf.predict_proba(X_train)[:,1]
xgb_pred_probabilties = model_xgb.predict_proba(X_train)[:,1]

In [ ]:
# Histogram of predicted probabilities ("risk estimates") from Logit Model with Elastic Net Regularization
plt.figure(figsize=(10,4))
plt.suptitle('Predicted probabilities from Logit Model with Elastic Net Regularization', fontsize=15)
plt.hist(logit_pred_probabilties[y_train==0], bins=50, label='Negatives')
plt.hist(logit_pred_probabilties[y_train==1], bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Probability of being Positive Class (y = 1)', fontsize=15)
plt.ylabel('Frequency', fontsize=20)
plt.legend(fontsize=10)
plt.tick_params(axis='both', labelsize=12, pad=5)
plt.show() 

In [ ]:
# Histogram of predicted probabilities ("risk estimates") from Random Forest Model
plt.figure(figsize=(10,4))
plt.suptitle('Predicted probabilities from Random Forest Model', fontsize=15)
plt.hist(rf_pred_probabilties[y_train==0], bins=50, label='Negatives')
plt.hist(rf_pred_probabilties[y_train==1], bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Probability of being Positive Class (y = 1)', fontsize=15)
plt.ylabel('Frequency', fontsize=20)
plt.legend(fontsize=10)
plt.tick_params(axis='both', labelsize=12, pad=5)
plt.show() 

In [ ]:
# Histogram of predicted probabilities ("risk estimates") from Gradient Boosted Forest Model
plt.figure(figsize=(10,4))
plt.suptitle('Predicted probabilities from Gradient Boosted Forest Model', fontsize=15)
plt.hist(xgb_pred_probabilties[y_train==0], bins=50, label='Negatives')
plt.hist(xgb_pred_probabilties[y_train==1], bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Probability of being Positive Class (y = 1)', fontsize=15)
plt.ylabel('Frequency', fontsize=20)
plt.legend(fontsize=10)
plt.tick_params(axis='both', labelsize=12, pad=5)
plt.show() 

In [ ]:
# Ensemble model for predicted probabilities (unweighted average as in the baseline paper)
average_pred_probabilities = (logit_pred_probabilties+
                              rf_pred_probabilties+
                              xgb_pred_probabilties)/3

average_pred_probabilities

In [ ]:
# Histogram of predicted probabilities ("risk estimates") from Ensemble Model
plt.figure(figsize=(10,4))
plt.suptitle('Predicted probabilities from Ensemble Model', fontsize=15)
plt.hist(average_pred_probabilities[y_train==0], bins=50, label='Negatives')
plt.hist(average_pred_probabilities[y_train==1], bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Probability of being Positive Class (y = 1)', fontsize=15)
plt.ylabel('Frequency', fontsize=20)
plt.legend(fontsize=10)
plt.tick_params(axis='both', labelsize=12, pad=5)
plt.show() 

In [ ]:
# Ensemble for predicted labels (class), i.e. 0 or 1
average_pred = (logit_pred+
                rf_pred+
                XGB_pred)/3

## Cross-Validation (2x7 k-fold)

In [ ]:
cv = RepeatedKFold(n_splits = 7, n_repeats = 2)

### Model 0 (Plain Logistic Regression)

In [ ]:
model_0_cv_roc_auc = cross_val_score(model0, X_train, y_train, scoring = "roc_auc", cv = cv)
model_0_cv_precision = cross_val_score(model0, X_train, y_train, scoring = "recall", cv = cv)
model_0_cv_brier = cross_val_score(model0, X_train, y_train, scoring = "neg_brier_score", cv = cv)

In [ ]:
# print(model_0_cv_roc_auc)
print(np.sum(model_0_cv_roc_auc) / len(model_0_cv_roc_auc))
# print(model_0_cv_precision)
print(np.sum(model_0_cv_precision) / len(model_0_cv_precision))
# print(model_0_cv_brier)
print(np.sum(model_0_cv_brier) / len(model_0_cv_brier))

0.8442572869016508
0.22739070389617597
-0.039891734126948186


### Model 1 (Logistic Regression with Elastic Net Regularization)

In [ ]:
model_1_cv_accuracy = cross_val_score(model1, X_train, y_train, scoring = "accuracy", cv = cv)
model_1_cv_roc_auc = cross_val_score(model1, X_train, y_train, scoring = "roc_auc", cv = cv)
model_1_cv_precision = cross_val_score(model1, X_train, y_train, scoring = "precision", cv = cv)
model_1_cv_recall = cross_val_score(model1, X_train, y_train, scoring = "recall", cv = cv)
#model_1_cv_brier = cross_val_score(model1, X_train, y_train, scoring = "neg_brier_score", cv = cv)

In [ ]:
print(np.sum(model_1_cv_accuracy) / len(model_1_cv_accuracy))
print(np.sum(model_1_cv_roc_auc) / len(model_1_cv_roc_auc))
print(np.sum(model_1_cv_precision) / len(model_1_cv_precision))
print(np.sum(model_1_cv_recall) / len(model_1_cv_recall))

### Model 2 (Random Forest)

In [ ]:
model_rf_cv_accuracy = cross_val_score(model_rf, X_train, y_train, scoring = "accuracy", cv = cv)
model_rf_cv_roc_auc = cross_val_score(model_rf, X_train, y_train, scoring = "roc_auc", cv = cv)
model_rf_cv_precision = cross_val_score(model_rf, X_train, y_train, scoring = "precision", cv = cv)
model_rf_cv_recall = cross_val_score(model_rf, X_train, y_train, scoring = "recall", cv = cv)
#model_rf_cv_brier = cross_val_score(model_rf, X_train, y_train, scoring = "neg_brier_score", cv = cv)

In [ ]:
print(np.sum(model_rf_cv_accuracy) / len(model_rf_cv_accuracy))
print(np.sum(model_rf_cv_roc_auc) / len(model_rf_cv_roc_auc))
print(np.sum(model_rf_cv_precision) / len(model_rf_cv_precision))
print(np.sum(model_rf_cv_recall) / len(model_rf_cv_recall))

### Model 3 (Gradient Boosted Forest)

In [ ]:
model_xgb_cv_accuracy = cross_val_score(model_xgb, X_train, y_train, scoring = "accuracy", cv = cv)
model_xgb_cv_roc_auc = cross_val_score(model_xgb, X_train, y_train, scoring = "roc_auc", cv = cv)
model_xgb_cv_precision = cross_val_score(model_xgb, X_train, y_train, scoring = "precision", cv = cv)
model_xgb_cv_recall = cross_val_score(model_xgb, X_train, y_train, scoring = "recall", cv = cv)
#model_xgb_cv_brier = cross_val_score(model_xgb, X_train, y_train, scoring = "neg_brier_score", cv = cv)

In [ ]:
print(np.sum(model_xgb_cv_accuracy) / len(model_xgb_cv_accuracy))
print(np.sum(model_xgb_cv_roc_auc) / len(model_xgb_cv_roc_auc))
print(np.sum(model_xgb_cv_precision) / len(model_xgb_cv_precision))
print(np.sum(model_xgb_cv_recall) / len(model_xgb_cv_recall))